In [ ]:
import time
import os
import numpy as np
from pynq import allocate
from pynq import MMIO
from pynq import Overlay

In [ ]:
os.getcwd()
ol = Overlay("dac5681.bit")

In [ ]:
# base addr
SPI_CONTROL_dac1  = 0x00_8000_0000
SPI_CONTROL_dac2  = 0x00_8001_0000
GPIO_7044_rst     = 0x00_8002_0000
SPI_CONTROL_7044  = 0x00_8003_0000
GPIO_dds_freq_1   = 0x00_8004_0000
GPIO_delay_1      = 0x00_8005_0000
GPIO_data_1       = 0x00_8006_0000
GPIO_sync_1       = 0x00_8007_0000
GPIO_en_1         = 0x00_8008_0000
GPIO_data_2       = 0x00_8009_0000
GPIO_dds_freq_2   = 0x00_800A_0000
GPIO_delay_2      = 0x00_800B_0000
GPIO_en_2         = 0x00_800C_0000
GPIO_sync_2       = 0x00_800D_0000
GPIO_adc_rst      = 0x00_800E_0000

# regs
gpio_ch1          = 0x0000
gpio_ch2          = 0x0008
soft_rst_n_reg    = 0x10
chip_reg          = 0x18
cpol_reg          = 0x20
cpha_reg          = 0x28
w_r_mode_reg      = 0x30
wr_width_reg      = 0x38
wr_data_reg       = 0x40
rd_width_reg      = 0x48
rd_target_num_reg = 0x50
wr_done_reg       = 0x58
rd_done_reg       = 0x68
rd_data_reg       = 0x78

# SPI从机列表(rd_mode=0-只读模式;rd_mode=1-指令控读模式)
chip_settings = {
    "HMC7044": {"chip_num":0, "cpol":0, "cpha":0, "wr_width":24, "rd_mode":1, "rd_comd_width":16, "rd_data_width":8},
    "DAC5681_1": {"chip_num":0, "cpol":0, "cpha":0, "wr_width":16, "rd_mode":1, "rd_comd_width":8, "rd_data_width":8},
    "DAC5681_2": {"chip_num":0, "cpol":0, "cpha":0, "wr_width":16, "rd_mode":1, "rd_comd_width":8, "rd_data_width":8},
}

In [ ]:
# 写寄存器
def write_reg(base_addr, offset, val):
    mmio = MMIO(base_addr, 64*1024)   # 创建一个名为mmio的MMIO对象,可用地址空间为64KB(与VIVADO设计中的地址分配相吻合)
    mmio.write(offset, val)           # 向地址为base_addr+offset的寄存器写入val


# 读寄存器
def read_reg(base_addr, offset):  
    mmio = MMIO(base_addr, 64*1024)
    read_val = mmio.read(offset)
    return read_val


# 写SPI
def spi_write(chip, wr_data):
    if chip in chip_settings:
        chip_num = chip_settings[chip]["chip_num"]
        cpol = chip_settings[chip]["cpol"]
        cpha = chip_settings[chip]["cpha"]
        wr_width = chip_settings[chip]["wr_width"]
        if chip == "HMC7044" :
            SPI_CONTROL = SPI_CONTROL_7044
        elif chip == "DAC5681_1" :
            SPI_CONTROL = SPI_CONTROL_dac1
        else :
            SPI_CONTROL = SPI_CONTROL_dac2
    write_reg(SPI_CONTROL, soft_rst_n_reg, 0x0)
    write_reg(SPI_CONTROL, chip_reg, chip_num)
    write_reg(SPI_CONTROL, cpol_reg, cpol)
    write_reg(SPI_CONTROL, cpha_reg, cpha)
    write_reg(SPI_CONTROL, w_r_mode_reg, 0x1)
    write_reg(SPI_CONTROL, wr_width_reg, wr_width)
    write_reg(SPI_CONTROL, wr_data_reg, wr_data)
    write_reg(SPI_CONTROL, soft_rst_n_reg, 0x1)
    wr_done = read_reg(SPI_CONTROL, wr_done_reg)
    while wr_done!=1 :
        wr_done = read_reg(SPI_CONTROL, wr_done_reg)


# 读SPI
def spi_read(chip, rd_command):
    if chip in chip_settings:
        chip_num = chip_settings[chip]["chip_num"]
        cpol = chip_settings[chip]["cpol"]
        cpha = chip_settings[chip]["cpha"]
        rd_mode = chip_settings[chip]["rd_mode"]
        rd_comd_width = chip_settings[chip]["rd_comd_width"]
        rd_data_width = chip_settings[chip]["rd_data_width"]
        if chip == "HMC7044" :
            SPI_CONTROL = SPI_CONTROL_7044
        elif chip == "DAC5681_1" :
            SPI_CONTROL = SPI_CONTROL_dac1
        else :
            SPI_CONTROL = SPI_CONTROL_dac2
    if rd_mode==0 :
        write_reg(SPI_CONTROL, soft_rst_n_reg, 0x0)
        write_reg(SPI_CONTROL, chip_reg, chip_num)
        write_reg(SPI_CONTROL, cpol_reg, cpol)
        write_reg(SPI_CONTROL, cpha_reg, cpha)
        write_reg(SPI_CONTROL, w_r_mode_reg, 0x0)
        write_reg(SPI_CONTROL, rd_width_reg, rd_data_width)
        write_reg(SPI_CONTROL, rd_target_num_reg, 0x1)
        write_reg(SPI_CONTROL, soft_rst_n_reg, 0x1)
    else :
        write_reg(SPI_CONTROL, soft_rst_n_reg, 0x0)
        write_reg(SPI_CONTROL, chip_reg, chip_num)
        write_reg(SPI_CONTROL, cpol_reg, cpol)
        write_reg(SPI_CONTROL, cpha_reg, cpha)
        write_reg(SPI_CONTROL, w_r_mode_reg, 0x2)
        write_reg(SPI_CONTROL, wr_width_reg, rd_comd_width)
        write_reg(SPI_CONTROL, wr_data_reg, rd_command)
        write_reg(SPI_CONTROL, rd_width_reg, rd_data_width)
        write_reg(SPI_CONTROL, rd_target_num_reg, 0x1)
        write_reg(SPI_CONTROL, soft_rst_n_reg, 0x1)
    rd_done = read_reg(SPI_CONTROL, rd_done_reg)
    while rd_done!=1 :
        rd_done = read_reg(SPI_CONTROL, rd_done_reg)
    rx_data = read_reg(SPI_CONTROL, rd_data_reg)
    rx_data_bin = format(rx_data, '032b')[-rd_data_width:]
    rx_data = int(rx_data_bin, 2)
    print(f"Successfully read, the result is {rd_data_width}'b{rx_data_bin}.")
    return rx_data

In [ ]:
################################################ dac5681 - 7044 ################################################
# Hard Reset
write_reg(GPIO_7044_rst, gpio_ch1, 0x1)
time.sleep(0.1)
write_reg(GPIO_7044_rst, gpio_ch1, 0x0)

# Soft Reset
spi_write("HMC7044", 0x000001)
time.sleep(0.1)
spi_write("HMC7044", 0x000000)

# Config RESERVED Reg
spi_write("HMC7044", 0x009600)
spi_write("HMC7044", 0x009700)
spi_write("HMC7044", 0x009800)
spi_write("HMC7044", 0x009900)
spi_write("HMC7044", 0x009A00)
spi_write("HMC7044", 0x009BAA)
spi_write("HMC7044", 0x009CAA)
spi_write("HMC7044", 0x009DAA)
spi_write("HMC7044", 0x009EAA)
spi_write("HMC7044", 0x009F55) # Update
spi_write("HMC7044", 0x00A056) # Update
spi_write("HMC7044", 0x00A197)
spi_write("HMC7044", 0x00A203)
spi_write("HMC7044", 0x00A300)
spi_write("HMC7044", 0x00A400)
spi_write("HMC7044", 0x00A500) # Update
spi_write("HMC7044", 0x00A61C)
spi_write("HMC7044", 0x00A700)
spi_write("HMC7044", 0x00A822) # Update
spi_write("HMC7044", 0x00A900)
spi_write("HMC7044", 0x00AB00)
spi_write("HMC7044", 0x00AC20)
spi_write("HMC7044", 0x00AD00)
spi_write("HMC7044", 0x00AE08)
spi_write("HMC7044", 0x00AF50)
spi_write("HMC7044", 0x00B009) # Update
spi_write("HMC7044", 0x00B10D)
spi_write("HMC7044", 0x00B200)
spi_write("HMC7044", 0x00B300)
spi_write("HMC7044", 0x00B500)
spi_write("HMC7044", 0x00B600)
spi_write("HMC7044", 0x00B700)
spi_write("HMC7044", 0x00B800)

# Config PLL2
spi_write("HMC7044", 0x000100)
spi_write("HMC7044", 0x000204)
spi_write("HMC7044", 0x00032E)
spi_write("HMC7044", 0x000448)
spi_write("HMC7044", 0x000540)
spi_write("HMC7044", 0x000600)
spi_write("HMC7044", 0x000700)
spi_write("HMC7044", 0x000901)
spi_write("HMC7044", 0x003101)
spi_write("HMC7044", 0x003201) # x2_bypass
spi_write("HMC7044", 0x003301) # R2_lsb
spi_write("HMC7044", 0x003400) # R2_msb
spi_write("HMC7044", 0x00352C) # N2_lsb
spi_write("HMC7044", 0x003601) # N2_msb
spi_write("HMC7044", 0x00370F) # cp_gain
spi_write("HMC7044", 0x003818)
spi_write("HMC7044", 0x003900)
spi_write("HMC7044", 0x003A00)
spi_write("HMC7044", 0x003B00)

# Config PLL1
spi_write("HMC7044", 0x000A17) # Config Input Buffer
spi_write("HMC7044", 0x000B17)
spi_write("HMC7044", 0x000C17)
spi_write("HMC7044", 0x000D17)
spi_write("HMC7044", 0x000E07)
spi_write("HMC7044", 0x0014E4) # Config PLL1
spi_write("HMC7044", 0x001503)
spi_write("HMC7044", 0x00160C)
spi_write("HMC7044", 0x001700)
spi_write("HMC7044", 0x001800)
spi_write("HMC7044", 0x001900)
spi_write("HMC7044", 0x001A08)
spi_write("HMC7044", 0x001B00)
spi_write("HMC7044", 0x001C04)
spi_write("HMC7044", 0x001D01)
spi_write("HMC7044", 0x001E04)
spi_write("HMC7044", 0x001F01)
spi_write("HMC7044", 0x002004)
spi_write("HMC7044", 0x002104)
spi_write("HMC7044", 0x002200)
spi_write("HMC7044", 0x002610)
spi_write("HMC7044", 0x002700)
spi_write("HMC7044", 0x00280C)
spi_write("HMC7044", 0x002904)
spi_write("HMC7044", 0x002A00)

# Config SYSREF
spi_write("HMC7044", 0x005A00)
spi_write("HMC7044", 0x005B06)
spi_write("HMC7044", 0x005C00)
spi_write("HMC7044", 0x005D01)

# Config GPIO
spi_write("HMC7044", 0x004600)
spi_write("HMC7044", 0x004700)
spi_write("HMC7044", 0x004800)
spi_write("HMC7044", 0x004900)
spi_write("HMC7044", 0x005036)
spi_write("HMC7044", 0x005132)
spi_write("HMC7044", 0x005200)
spi_write("HMC7044", 0x005300)
spi_write("HMC7044", 0x005403)

# Config Clock Distribution Network
spi_write("HMC7044", 0x006400)
spi_write("HMC7044", 0x006500)

# Config Masks
spi_write("HMC7044", 0x007000)
spi_write("HMC7044", 0x007110)

# Config Output Channel
spi_write("HMC7044", 0x00C8F2) # channal 0
spi_write("HMC7044", 0x00C904) # div_lsb
spi_write("HMC7044", 0x00CA00) # div_msb
spi_write("HMC7044", 0x00CB00)
spi_write("HMC7044", 0x00CC00)
spi_write("HMC7044", 0x00CD00)
spi_write("HMC7044", 0x00CE00)
spi_write("HMC7044", 0x00CF00)
spi_write("HMC7044", 0x00D001)

spi_write("HMC7044", 0x00D2FC) # channal 1
spi_write("HMC7044", 0x00D300) # div_lsb
spi_write("HMC7044", 0x00D401) # div_msb
spi_write("HMC7044", 0x00D500)
spi_write("HMC7044", 0x00D600)
spi_write("HMC7044", 0x00D700)
spi_write("HMC7044", 0x00D800)
spi_write("HMC7044", 0x00D900)
spi_write("HMC7044", 0x00DA30)

spi_write("HMC7044", 0x00DCF2) # channal 2
spi_write("HMC7044", 0x00DD18) # div_lsb
spi_write("HMC7044", 0x00DE00) # div_msb
spi_write("HMC7044", 0x00DF00)
spi_write("HMC7044", 0x00E000)
spi_write("HMC7044", 0x00E100)
spi_write("HMC7044", 0x00E200)
spi_write("HMC7044", 0x00E300)
spi_write("HMC7044", 0x00E411)

spi_write("HMC7044", 0x00E6F0) # channal 3
spi_write("HMC7044", 0x00E718) # div_lsb
spi_write("HMC7044", 0x00E800) # div_msb
spi_write("HMC7044", 0x00E900)
spi_write("HMC7044", 0x00EA00)
spi_write("HMC7044", 0x00EB00)
spi_write("HMC7044", 0x00EC00)
spi_write("HMC7044", 0x00ED00)
spi_write("HMC7044", 0x00EE30)

spi_write("HMC7044", 0x00F0F2) # channel 4
spi_write("HMC7044", 0x00F103) # div_lsb
spi_write("HMC7044", 0x00F200) # div_msb
spi_write("HMC7044", 0x00F300)
spi_write("HMC7044", 0x00F400)
spi_write("HMC7044", 0x00F500)
spi_write("HMC7044", 0x00F600)
spi_write("HMC7044", 0x00F700)
spi_write("HMC7044", 0x00F811)

spi_write("HMC7044", 0x00FAF0) # channel 5
spi_write("HMC7044", 0x00FB00) # div_lsb
spi_write("HMC7044", 0x00FC06) # div_msb
spi_write("HMC7044", 0x00FD00)
spi_write("HMC7044", 0x00FE00)
spi_write("HMC7044", 0x00FF00)
spi_write("HMC7044", 0x010000)
spi_write("HMC7044", 0x010100)
spi_write("HMC7044", 0x010230)

spi_write("HMC7044", 0x0104F3) # channal 6 - 1GHz
spi_write("HMC7044", 0x010503) # div_lsb
spi_write("HMC7044", 0x010600) # div_msb
spi_write("HMC7044", 0x010700)
spi_write("HMC7044", 0x010800)
spi_write("HMC7044", 0x010900)
spi_write("HMC7044", 0x010A00)
spi_write("HMC7044", 0x010B00)
spi_write("HMC7044", 0x010C11)

spi_write("HMC7044", 0x010EF1) # channal 7 - 1GHz
spi_write("HMC7044", 0x010F03) # div_lsb
spi_write("HMC7044", 0x011000) # div_msb
spi_write("HMC7044", 0x011100)
spi_write("HMC7044", 0x011200)
spi_write("HMC7044", 0x011300)
spi_write("HMC7044", 0x011400)
spi_write("HMC7044", 0x011502)
spi_write("HMC7044", 0x011630)

spi_write("HMC7044", 0x0118F2) # channal 8
spi_write("HMC7044", 0x011918) # div_lsb
spi_write("HMC7044", 0x011A00) # div_msb
spi_write("HMC7044", 0x011B00)
spi_write("HMC7044", 0x011C00)
spi_write("HMC7044", 0x011D00)
spi_write("HMC7044", 0x011E00)
spi_write("HMC7044", 0x011F00)
spi_write("HMC7044", 0x012011)

spi_write("HMC7044", 0x0122FC) # channal 9
spi_write("HMC7044", 0x012300) # div_lsb
spi_write("HMC7044", 0x012406) # div_msb
spi_write("HMC7044", 0x012500)
spi_write("HMC7044", 0x012600)
spi_write("HMC7044", 0x012700)
spi_write("HMC7044", 0x012800)
spi_write("HMC7044", 0x012900)
spi_write("HMC7044", 0x012A30)

spi_write("HMC7044", 0x012CF2) # channal 10
spi_write("HMC7044", 0x012D02) # div_lsb
spi_write("HMC7044", 0x012E00) # div_msb
spi_write("HMC7044", 0x012F00)
spi_write("HMC7044", 0x013000)
spi_write("HMC7044", 0x01310F)
spi_write("HMC7044", 0x013200)
spi_write("HMC7044", 0x013300)
spi_write("HMC7044", 0x013401)

spi_write("HMC7044", 0x0136FC) # channal 11
spi_write("HMC7044", 0x013700) # div_lsb
spi_write("HMC7044", 0x013801) # div_msb
spi_write("HMC7044", 0x013900)
spi_write("HMC7044", 0x013A00)
spi_write("HMC7044", 0x013B00)
spi_write("HMC7044", 0x013C00)
spi_write("HMC7044", 0x013D00)
spi_write("HMC7044", 0x013E30)

spi_write("HMC7044", 0x0140F3) # channel 12 - 250MHz
spi_write("HMC7044", 0x01410C) # div_lsb
spi_write("HMC7044", 0x014200) # div_msb
spi_write("HMC7044", 0x014300)
spi_write("HMC7044", 0x014400)
spi_write("HMC7044", 0x014500)
spi_write("HMC7044", 0x014600)
spi_write("HMC7044", 0x014700)
spi_write("HMC7044", 0x014811)

spi_write("HMC7044", 0x014AFC) # channel 13
spi_write("HMC7044", 0x014B00) # div_lsb
spi_write("HMC7044", 0x014C01) # div_msb
spi_write("HMC7044", 0x014D00)
spi_write("HMC7044", 0x014E00)
spi_write("HMC7044", 0x014F00)
spi_write("HMC7044", 0x015000)
spi_write("HMC7044", 0x015100)
spi_write("HMC7044", 0x015230)
time.sleep(0.1)

# Restart dividers/FSMs
spi_write("HMC7044", 0x000102)
time.sleep(0.1)
spi_write("HMC7044", 0x000100)
time.sleep(0.1)
print("Successfully config.")

In [ ]:
###############################################################DAC1###############################################################

In [ ]:
# 设置DDS1信号频率
write_reg(GPIO_data_1, gpio_ch2, 0x0)
rd = read_reg(GPIO_dds_freq_1, gpio_ch2)
while rd!=1 :
    rd = read_reg(GPIO_dds_freq_1, gpio_ch2)
write_reg(GPIO_dds_freq_1, gpio_ch1, 1311) # 20MHz

# 设置直流量
# write_reg(GPIO_data_1, gpio_ch2, 0x1)
# write_reg(GPIO_data_1, gpio_ch1, 0x7FFF)

In [ ]:
################################################ dac5681_1(内置SYNC)################################################
# 关闭外置SYNC
write_reg(GPIO_sync_1, gpio_ch1, 0x0)

# Hard Reset
write_reg(GPIO_adc_rst, gpio_ch1, 0x0)
time.sleep(1)
write_reg(GPIO_adc_rst, gpio_ch1, 0x1)

# 配置DAC
spi_write("DAC5681_1", 0x0110) # {2'b DAC_delay, 2'b01;                                            SLFTST_ena, 3'b FIFO_offset}
spi_write("DAC5681_1", 0x02C0) # {1'b Twos_comp, 3'b100;                                           4'b0000}
spi_write("DAC5681_1", 0x0301) # {DAC_offset_en, SLFTST_err_mask, FIFO_err_mask, Pattern_err_mask; 2'b00, SW_sync, SW_sync_sel}
spi_write("DAC5681_1", 0x0400) # {1'b0, SLFTST_err, FIFO_err, Pattern_err;                         4'b0000}
spi_write("DAC5681_1", 0x0500) # {SIF4, rev_bus, clkdiv_sync_dis, 1'b0;                            1'b0, DLL_bypass, 2'b0}
spi_write("DAC5681_1", 0x060C) # {3'b0, Sleep_A;                                                   BiasLPF_A, 2'b10, DLL_sleep}
spi_write("DAC5681_1", 0x07FF) # {4'b DACA_gain;                                                   4'b1111}
spi_write("DAC5681_1", 0x0804) # {4'b0000;                                                         1'b0, DLL_restart, 2'b0}
spi_write("DAC5681_1", 0x0900) # Reserved
spi_write("DAC5681_1", 0x0AC0) # {4'b DLL_delay;                                                   DLL_invclk, 3'b DLL_ifixed}
spi_write("DAC5681_1", 0x0B00) # Reserved
spi_write("DAC5681_1", 0x0C00) # {2'b0, Offset_sync, 5'b OffsetA(12:8)}
spi_write("DAC5681_1", 0x0D00) # {8'b OffsetA(7:0)}
spi_write("DAC5681_1", 0x0E00) # {3'b SDO_func_sel, 1'b0;                                          4'b0000}
spi_write("DAC5681_1", 0x0F00) # Reserved

# 开启DCLK
write_reg(GPIO_en_1, gpio_ch1, 0x0)
time.sleep(0.001)

# 等待DAC-DLL锁定
spi_write("DAC5681_1", 0x0800) # {4'b0000;                                                         1'b0, DLL_restart, 2'b0}
status0 = spi_read("DAC5681_1", (1<<7)|(0x00&0x1F)) # 等待DLL锁定
dll_lock = (status0 & 0x40) >> 6
while dll_lock!=1 :
    status0 = spi_read("DAC5681_1", (1<<7)|(0x00&0x1F))
    dll_lock = (status0 & 0x40) >> 6

# 启动内置SYNC    
spi_write("DAC5681_1", 0x0303) 

# 启动LVDS数据
write_reg(GPIO_en_1, gpio_ch2, 0x0)

In [ ]:
################################################ dac5681_1(外置SYNC)################################################
# 关闭外置SYNC
write_reg(GPIO_sync_1, gpio_ch1, 0x0)

# Hard Reset
write_reg(GPIO_adc_rst, gpio_ch1, 0x0)
time.sleep(1)
write_reg(GPIO_adc_rst, gpio_ch1, 0x1)

# 配置DAC
spi_write("DAC5681_1", 0x0110) # {2'b DAC_delay, 2'b01;                                            SLFTST_ena, 3'b FIFO_offset}
spi_write("DAC5681_1", 0x02C0) # {1'b Twos_comp, 3'b100;                                           4'b0000}
spi_write("DAC5681_1", 0x0300) # {DAC_offset_en, SLFTST_err_mask, FIFO_err_mask, Pattern_err_mask; 2'b00, SW_sync, SW_sync_sel}
spi_write("DAC5681_1", 0x0400) # {1'b0, SLFTST_err, FIFO_err, Pattern_err;                         4'b0000}
spi_write("DAC5681_1", 0x0500) # {SIF4, rev_bus, clkdiv_sync_dis, 1'b0;                            1'b0, DLL_bypass, 2'b0}
spi_write("DAC5681_1", 0x060C) # {3'b0, Sleep_A;                                                   BiasLPF_A, 2'b10, DLL_sleep}
spi_write("DAC5681_1", 0x07FF) # {4'b DACA_gain;                                                   4'b1111}
spi_write("DAC5681_1", 0x0804) # {4'b0000;                                                         1'b0, DLL_restart, 2'b0}
spi_write("DAC5681_1", 0x0900) # Reserved
spi_write("DAC5681_1", 0x0AC0) # {4'b DLL_delay;                                                   DLL_invclk, 3'b DLL_ifixed}
spi_write("DAC5681_1", 0x0B00) # Reserved
spi_write("DAC5681_1", 0x0C00) # {2'b0, Offset_sync, 5'b OffsetA(12:8)}
spi_write("DAC5681_1", 0x0D00) # {8'b OffsetA(7:0)}
spi_write("DAC5681_1", 0x0E00) # {3'b SDO_func_sel, 1'b0;                                          4'b0000}
spi_write("DAC5681_1", 0x0F00) # Reserved

# 开启DCLK
write_reg(GPIO_en_1, gpio_ch1, 0x0)
time.sleep(0.001)

# 等待DAC-DLL锁定
spi_write("DAC5681_1", 0x0800) # {4'b0000;                                                         1'b0, DLL_restart, 2'b0}
status0 = spi_read("DAC5681_1", (1<<7)|(0x00&0x1F)) # 等待DLL锁定
dll_lock = (status0 & 0x40) >> 6
while dll_lock!=1 :
    status0 = spi_read("DAC5681_1", (1<<7)|(0x00&0x1F))
    dll_lock = (status0 & 0x40) >> 6

# 启动外置SYNC
write_reg(GPIO_sync_1, gpio_ch1, 0x2)

# 启动LVDS数据
write_reg(GPIO_en_1, gpio_ch2, 0x0)

In [ ]:
# 调节FPGA端DAC1-DCLK的odelay
delay_value = 0
rd = read_reg(GPIO_delay_1, gpio_ch2)&0x01
while rd!=1 :
    rd = read_reg(GPIO_delay_1, gpio_ch2)&0x01
write_reg(GPIO_delay_1, gpio_ch1, delay_value)
delay_monitor = read_reg(GPIO_delay_1, gpio_ch2)>>1
print(f"The delay value of clk now is {delay_monitor}.")

In [ ]:
###############################################################DAC2###############################################################

In [ ]:
# 设置DDS2信号频率
write_reg(GPIO_data_2, gpio_ch2, 0x0)
rd = read_reg(GPIO_dds_freq_2, gpio_ch2)
while rd!=1 :
    rd = read_reg(GPIO_dds_freq_2, gpio_ch2)
write_reg(GPIO_dds_freq_2, gpio_ch1, 1311) # 20MHz

# 设置直流量
# write_reg(GPIO_data_2, gpio_ch2, 0x1)
# write_reg(GPIO_data_2, gpio_ch1, 0x7FFF)

In [ ]:
################################################ dac5681_2(内置SYNC)################################################
# 关闭外置SYNC
write_reg(GPIO_sync_2, gpio_ch1, 0x0)

# Hard Reset
write_reg(GPIO_adc_rst, gpio_ch2, 0x0)
time.sleep(1)
write_reg(GPIO_adc_rst, gpio_ch2, 0x1)

# 配置DAC
spi_write("DAC5681_2", 0x0110) # {2'b DAC_delay, 2'b01;                                            SLFTST_ena, 3'b FIFO_offset}
spi_write("DAC5681_2", 0x02C0) # {1'b Twos_comp, 3'b100;                                           4'b0000}
spi_write("DAC5681_2", 0x0301) # {DAC_offset_en, SLFTST_err_mask, FIFO_err_mask, Pattern_err_mask; 2'b00, SW_sync, SW_sync_sel}
spi_write("DAC5681_1", 0x0400) # {1'b0, SLFTST_err, FIFO_err, Pattern_err;                         4'b0000}
spi_write("DAC5681_2", 0x0500) # {SIF4, rev_bus, clkdiv_sync_dis, 1'b0;                            1'b0, DLL_bypass, 2'b0}
spi_write("DAC5681_2", 0x060C) # {3'b0, Sleep_A;                                                   BiasLPF_A, 2'b10, DLL_sleep}
spi_write("DAC5681_2", 0x07FF) # {4'b DACA_gain;                                                   4'b1111}
spi_write("DAC5681_2", 0x0804) # {4'b0000;                                                         1'b0, DLL_restart, 2'b0}
spi_write("DAC5681_2", 0x0900) # Reserved
spi_write("DAC5681_2", 0x0AC0) # {4'b DLL_delay;                                                   DLL_invclk, 3'b DLL_ifixed}
spi_write("DAC5681_2", 0x0B00) # Reserved
spi_write("DAC5681_2", 0x0C00) # {2'b0, Offset_sync, 5'b OffsetA(12:8)}
spi_write("DAC5681_2", 0x0D00) # {8'b OffsetA(7:0)}
spi_write("DAC5681_2", 0x0E00) # {3'b SDO_func_sel, 1'b0;                                          4'b0000}
spi_write("DAC5681_2", 0x0F00) # Reserved

# 开启DCLK
write_reg(GPIO_en_2, gpio_ch1, 0x0)
time.sleep(0.001)

# 等待DAC-DLL锁定
spi_write("DAC5681_2", 0x0800) # {4'b0000;                                                         1'b0, DLL_restart, 2'b0}
status0 = spi_read("DAC5681_2", (1<<7)|(0x00&0x1F)) # 等待DLL锁定
dll_lock = (status0 & 0x40) >> 6
while dll_lock!=1 :
    status0 = spi_read("DAC5681_2", (1<<7)|(0x00&0x1F))
    dll_lock = (status0 & 0x40) >> 6

# 启动内置SYNC    
spi_write("DAC5681_2", 0x0303) 

# 启动LVDS数据
write_reg(GPIO_en_2, gpio_ch2, 0x0)

In [ ]:
################################################ dac5681_2(外置SYNC)################################################
# 关闭外置SYNC
write_reg(GPIO_sync_2, gpio_ch1, 0x0)

# Hard Reset
write_reg(GPIO_adc_rst, gpio_ch2, 0x0)
time.sleep(1)
write_reg(GPIO_adc_rst, gpio_ch2, 0x1)

# 配置DAC
spi_write("DAC5681_2", 0x0110) # {2'b DAC_delay, 2'b01;                                            SLFTST_ena, 3'b FIFO_offset}
spi_write("DAC5681_2", 0x02C0) # {1'b Twos_comp, 3'b100;                                           4'b0000}
spi_write("DAC5681_2", 0x0300) # {DAC_offset_en, SLFTST_err_mask, FIFO_err_mask, Pattern_err_mask; 2'b00, SW_sync, SW_sync_sel}
spi_write("DAC5681_1", 0x0400) # {1'b0, SLFTST_err, FIFO_err, Pattern_err;                         4'b0000}
spi_write("DAC5681_2", 0x0500) # {SIF4, rev_bus, clkdiv_sync_dis, 1'b0;                            1'b0, DLL_bypass, 2'b0}
spi_write("DAC5681_2", 0x060C) # {3'b0, Sleep_A;                                                   BiasLPF_A, 2'b10, DLL_sleep}
spi_write("DAC5681_2", 0x07FF) # {4'b DACA_gain;                                                   4'b1111}
spi_write("DAC5681_2", 0x0804) # {4'b0000;                                                         1'b0, DLL_restart, 2'b0}
spi_write("DAC5681_2", 0x0900) # Reserved
spi_write("DAC5681_2", 0x0AC0) # {4'b DLL_delay;                                                   DLL_invclk, 3'b DLL_ifixed}
spi_write("DAC5681_2", 0x0B00) # Reserved
spi_write("DAC5681_2", 0x0C00) # {2'b0, Offset_sync, 5'b OffsetA(12:8)}
spi_write("DAC5681_2", 0x0D00) # {8'b OffsetA(7:0)}
spi_write("DAC5681_2", 0x0E00) # {3'b SDO_func_sel, 1'b0;                                          4'b0000}
spi_write("DAC5681_2", 0x0F00) # Reserved

# 开启DCLK
write_reg(GPIO_en_2, gpio_ch1, 0x0)
time.sleep(0.001)

# 等待DAC-DLL锁定
spi_write("DAC5681_2", 0x0800) # {4'b0000;                                                         1'b0, DLL_restart, 2'b0}
status0 = spi_read("DAC5681_2", (1<<7)|(0x00&0x1F)) # 等待DLL锁定
dll_lock = (status0 & 0x40) >> 6
while dll_lock!=1 :
    status0 = spi_read("DAC5681_2", (1<<7)|(0x00&0x1F))
    dll_lock = (status0 & 0x40) >> 6

# 启动外置SYNC
write_reg(GPIO_sync_2, gpio_ch1, 0x2)

# 启动LVDS数据
write_reg(GPIO_en_2, gpio_ch2, 0x0)

In [ ]:
# 调节FPGA端DAC2-DCLK的odelay
delay_value = 0
rd = read_reg(GPIO_delay_2, gpio_ch2)&0x01
while rd!=1 :
    rd = read_reg(GPIO_delay_2, gpio_ch2)&0x01
write_reg(GPIO_delay_2, gpio_ch1, delay_value)
delay_monitor = read_reg(GPIO_delay_2, gpio_ch2)>>1
print(f"The delay value of clk now is {delay_monitor}.")